# Tool Interception -- Contracts, Identity & Field Filtering

This notebook demonstrates SafeAI's **action-boundary interceptor** -- the layer that
enforces policy on every tool call (request and response).  It covers:

- **Tool contracts** -- declaring what a tool accepts and emits (tags + fields).
- **Agent identity** -- binding agents to specific tools and clearance tags.
- **Field-level filtering** -- stripping parameters/response fields that fall outside a contract.

Because the interceptor needs contracts and identity files on disk, we use
`SafeAI.from_config()` to load everything from a temporary working directory
scaffolded by `safeai init`.

In [1]:
import tempfile, yaml
from pathlib import Path
from click.testing import CliRunner
from safeai.cli.init import init_command
from safeai import SafeAI

# Scaffold a fresh SafeAI workspace in a temp directory
tmp = tempfile.TemporaryDirectory()
work = Path(tmp.name)
result = CliRunner().invoke(init_command, ["--path", str(work)])
print(result.output)

# Remove the default example contract (it declares send_email, we'll write our own)
example_contract = work / "contracts" / "example.yaml"
if example_contract.exists():
    example_contract.unlink()

# ---- Write a custom tool contract for "send_email" ----
contract_doc = {
    "version": "v1alpha1",
    "contract": {
        "tool_name": "send_email",
        "description": "Sends email notifications to approved recipients.",
        "accepts": {
            "tags": ["internal"],
            "fields": ["to", "subject", "body"],
        },
        "emits": {
            "tags": ["internal"],
            "fields": ["status", "message_id"],
        },
        "stores": {
            "fields": ["to", "subject", "sent_at"],
            "retention": "30d",
        },
        "side_effects": {
            "reversible": False,
            "requires_approval": False,
            "description": "External message delivery is irreversible.",
        },
    },
}
(work / "contracts" / "send_email.yaml").write_text(yaml.dump(contract_doc))

# ---- Write a custom agent identity ----
identity_doc = {
    "version": "v1alpha1",
    "agent": {
        "agent_id": "notebook-agent",
        "description": "Demo agent used in this notebook.",
        "tools": ["send_email"],
        "clearance_tags": ["internal", "personal"],
    },
}
(work / "agents" / "notebook.yaml").write_text(yaml.dump(identity_doc))

# Load SafeAI from the workspace
ai = SafeAI.from_config(work / "safeai.yaml")
print("SafeAI loaded with contracts + agent identity.")

SafeAI initialized
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/safeai.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/policies/default.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/contracts/example.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/schemas/memory.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/agents/default.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/plugins/example.py
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/tenants/policy-sets.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmp4tyxlqwj/alerts/default.yaml

SafeAI loaded with contracts + agent identity.


## 1. Tool contract enforcement

When a tool call arrives, the interceptor first checks whether a **contract** exists
for the requested tool, and whether the request's `data_tags` are accepted by that
contract.  If both pass, the call is allowed and `filtered_params` contains the
parameters pruned to the contract's declared `accepts.fields`.

In [2]:
# Valid request -- all params match the contract's accepted fields,
# and the data tag "internal" is accepted by the send_email contract.
result = ai.intercept_tool_request(
    tool_name="send_email",
    parameters={"to": "alice@example.com", "subject": "Hello", "body": "Hi there!"},
    data_tags=["internal"],
    agent_id="notebook-agent",
    session_id="demo-session-1",
)

print("Decision action :", result.decision.action)
print("Filtered params :", result.filtered_params)
print("Stripped fields  :", result.stripped_fields)
print("Unauthorized tags:", result.unauthorized_tags)

Decision action : allow
Filtered params : {'to': 'alice@example.com', 'subject': 'Hello', 'body': 'Hi there!'}
Stripped fields  : []
Unauthorized tags: []


## 2. Undeclared tool is blocked

If no contract is registered for a tool, the interceptor blocks the request
outright.  This is the **deny-by-default** posture: every tool must have a
contract before it can be invoked.

In [3]:
# No contract exists for "run_sql" -- should be blocked
result = ai.intercept_tool_request(
    tool_name="run_sql",
    parameters={"query": "SELECT * FROM users"},
    data_tags=["internal"],
    agent_id="notebook-agent",
    session_id="demo-session-1",
)

print("Decision action:", result.decision.action)
print("Reason         :", result.decision.reason)
print("Policy name    :", result.decision.policy_name)

Decision action: block
Reason         : tool 'run_sql' has no declared contract
Policy name    : tool-contract


## 3. Field-level filtering

When a request includes parameters that are **not** listed in the contract's
`accepts.fields`, those extra fields are silently stripped.  The caller receives
`filtered_params` with only the allowed keys, and `stripped_fields` lists what
was removed.

In [4]:
# Request includes extra fields: "cc" and "priority" are not in the contract
result = ai.intercept_tool_request(
    tool_name="send_email",
    parameters={
        "to": "bob@example.com",
        "subject": "Update",
        "body": "Status report attached.",
        "cc": "manager@example.com",      # not in contract
        "priority": "high",                # not in contract
    },
    data_tags=["internal"],
    agent_id="notebook-agent",
    session_id="demo-session-1",
)

print("Decision action :", result.decision.action)
print("Filtered params :", result.filtered_params)
print("Stripped fields  :", result.stripped_fields)
print()
print("Only 'to', 'subject', 'body' survive -- 'cc' and 'priority' are stripped.")

Decision action : allow
Filtered params : {'to': 'bob@example.com', 'subject': 'Update', 'body': 'Status report attached.'}
Stripped fields  : ['cc', 'priority']

Only 'to', 'subject', 'body' survive -- 'cc' and 'priority' are stripped.


## 4. Agent identity -- tool binding

Each agent identity declares which tools it is allowed to use via the `tools`
list.  If an agent tries to invoke a tool that is **not** in its binding list,
the request is blocked by the identity layer, even if a contract exists for that
tool.

In [5]:
# "notebook-agent" is only bound to ["send_email"].
# Trying to call "delete_user" should be blocked by identity.
result_blocked = ai.intercept_tool_request(
    tool_name="delete_user",
    parameters={"user_id": "u-42"},
    data_tags=["internal"],
    agent_id="notebook-agent",
    session_id="demo-session-1",
)

print("--- delete_user (not bound) ---")
print("Decision action:", result_blocked.decision.action)
print("Reason         :", result_blocked.decision.reason)
print("Policy name    :", result_blocked.decision.policy_name)
print()

# Calling "send_email" should still work fine.
result_allowed = ai.intercept_tool_request(
    tool_name="send_email",
    parameters={"to": "carol@example.com", "subject": "Test", "body": "OK"},
    data_tags=["internal"],
    agent_id="notebook-agent",
    session_id="demo-session-1",
)

print("--- send_email (bound) ---")
print("Decision action:", result_allowed.decision.action)
print("Filtered params:", result_allowed.filtered_params)

--- delete_user (not bound) ---
Decision action: block
Reason         : tool 'delete_user' has no declared contract
Policy name    : tool-contract

--- send_email (bound) ---
Decision action: allow
Filtered params: {'to': 'carol@example.com', 'subject': 'Test', 'body': 'OK'}


## 5. Agent clearance tags

Beyond tool binding, each agent identity also declares **clearance tags** -- the
data classification levels it is allowed to handle.  If a request carries a data
tag that exceeds the agent's clearance, the interceptor blocks the call and
reports the `unauthorized_tags`.

In [6]:
# "notebook-agent" has clearance for ["internal", "personal"] but NOT "secret".
result = ai.intercept_tool_request(
    tool_name="send_email",
    parameters={"to": "dave@example.com", "subject": "Classified", "body": "..."},
    data_tags=["secret"],
    agent_id="notebook-agent",
    session_id="demo-session-1",
)

print("Decision action :", result.decision.action)
print("Reason          :", result.decision.reason)
print("Unauthorized tags:", result.unauthorized_tags)
print("Policy name     :", result.decision.policy_name)

Decision action : block
Reason          : tool 'send_email' does not accept data tags: secret
Unauthorized tags: ['secret']
Policy name     : tool-contract


## 6. Response filtering

The interceptor also enforces contracts on tool **responses**.  Any output fields
that are not listed in the contract's `emits.fields` are stripped before they
reach the calling agent.  This prevents tools from leaking undeclared data.

In [7]:
# Simulate a tool response that includes extra fields beyond what the
# contract declares in emits.fields ("status" and "message_id" only).
result = ai.intercept_tool_response(
    tool_name="send_email",
    response={
        "status": "sent",
        "message_id": "msg-abc-123",
        "internal_trace_id": "trace-xyz-789",   # not in emits
        "server_ip": "10.0.0.42",               # not in emits
    },
    agent_id="notebook-agent",
    request_data_tags=["internal"],
    session_id="demo-session-1",
)

print("Decision action  :", result.decision.action)
print("Filtered response:", result.filtered_response)
print("Stripped fields   :", result.stripped_fields)
print()
print("Only 'status' and 'message_id' survive -- undeclared fields are removed.")

Decision action  : redact
Filtered response: {'status': 'sent', 'message_id': 'msg-abc-123'}
Stripped fields   : ['internal_trace_id', 'server_ip']

Only 'status' and 'message_id' survive -- undeclared fields are removed.


In [8]:
# Cleanup
tmp.cleanup()
print("Temporary workspace cleaned up.")

Temporary workspace cleaned up.


---

### Summary

| Layer | What it enforces | Block trigger |
|---|---|---|
| **Tool contract** | Tool must be declared; request data tags must be accepted | No contract, or unauthorized tags |
| **Agent identity** | Agent must be bound to the tool; clearance must cover data tags | Tool not in binding, or tag exceeds clearance |
| **Field filtering (request)** | Only `accepts.fields` pass through | Extra params stripped silently |
| **Field filtering (response)** | Only `emits.fields` pass through | Extra response keys stripped silently |

All decisions are recorded in the audit log so you can trace exactly why a call
was allowed, filtered, or blocked.